# Z3-solver

In [ ]:
!pip install z3-solver

In [ ]:
import z3
from z3 import *

In [ ]:
x1 = Int('x1')
x2 = Int('x2')
folha1 = And(x1 < 2, x2 < 5)

res = Real('Out')
imp = Implies(folha1, res == 1.2)

In [ ]:
valor = imp.arg(1).arg(1)
valor

6/5

In [ ]:
float(valor.as_decimal(1)) * 2

2.4

In [ ]:
x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


In [ ]:
x = Int('x')
y = Int('y')
print (simplify(x + y + 2*x + 3))
print (simplify(x < y + x + 2))
print (simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)))

3 + 3*x + y
Not(y <= -2)
And(x >= 2, 2*x**2 + y**2 >= 3)


In [ ]:
x = Int('x')
y = Int('y')
print (x**2 + y**2 >= 1)
print (x**2 + y**2 >= 1)

x**2 + y**2 >= 1
x**2 + y**2 >= 1


In [ ]:
x = Int('x')
y = Int('y')
n = x + y >= 3
print ("num args: ", n.num_args())
print ("children: ", n.children())
print ("1st child:", n.arg(0))
print ("2nd child:", n.arg(1))
print ("operator: ", n.decl())
print ("op name:  ", n.decl().name())

num args:  2
children:  [x + y, 3]
1st child: x + y
2nd child: 3
operator:  >=
op name:   >=


In [ ]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 > 3, x**3 + y < 5)

[y = 2, x = 0]


In [ ]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

set_option(precision=30)
print ("Solving, and displaying result with 30 decimal places")
solve(x**2 + y**2 == 3, x**3 == 2)

[y = -1.188528059421316533710369365015?,
 x = 1.259921049894873164767210607278?]
Solving, and displaying result with 30 decimal places
[y = -1.188528059421316533710369365015?,
 x = 1.259921049894873164767210607278?]


In [ ]:
print (1/3)
print (RealVal(1)/3)
print (Q(1,3))

x = Real('x')
print (x + 1/3)
print (x + Q(1,3))
print (x + "1/3")
print (x + 0.25)

0.3333333333333333
1/3
0.333333333333333333333333333333?
x + 0.3333333333333333
x + 0.333333333333333333333333333333?
x + 0.333333333333333333333333333333?
x + 0.25


In [ ]:
x = Real('x')
solve(3*x == 1)

set_option(rational_to_decimal=True)
solve(3*x == 1)

set_option(precision=30)
solve(3*x == 1)

[x = 0.333333333333333333333333333333?]
[x = 0.333333333333333333333333333333?]
[x = 0.333333333333333333333333333333?]


In [ ]:
x = Real('x')
solve(x > 4, x < 0)

no solution


In [ ]:
p = Bool('p')
q = Bool('q')
r = Bool('r')
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

[q = True, p = False, r = False]


# imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_wine

In [ ]:
eps = np.finfo(np.float32).eps

In [ ]:
eps

1.1920929e-07

In [ ]:
# prompt: gere uma matriz aleatorio 2 por 10 "predictions"

predictions = np.random.rand(2, 10)
print(predictions)

[[0.42004969 0.17905328 0.96319511 0.64090213 0.49353736 0.56770813
  0.25761041 0.09066835 0.23626732 0.1731601 ]
 [0.01509162 0.09205328 0.0180217  0.09686344 0.70678759 0.79202099
  0.86461317 0.2055357  0.35605704 0.13023163]]


In [ ]:
predictions = np.clip(predictions, eps, 1 - eps, dtype=np.float64)

In [ ]:
predictions

array([[0.42004969, 0.17905328, 0.96319511, 0.64090213, 0.49353736,
        0.56770813, 0.25761041, 0.09066835, 0.23626732, 0.1731601 ],
       [0.01509162, 0.09205328, 0.0180217 , 0.09686344, 0.70678759,
        0.79202099, 0.86461317, 0.2055357 , 0.35605704, 0.13023163]])

In [ ]:
# loss.link.link(predictions)

# Functions

In [ ]:
def multiclass_initial_prediction(y, n_classes):
    """Calcula a previsão inicial para cada classe no caso multiclasse."""
    class_counts = np.bincount(y, minlength=n_classes)
    class_probs = class_counts / len(y)  # Frequências das classes
    print(class_probs)

    # Calcula a log odds para cada classe
    log_odds = np.log(class_probs + 1e-15)  # Evita log(0) com pequeno valor
    print(log_odds)
    print("final:", log_odds - log_odds.mean())
    return log_odds - log_odds.mean()  # Centraliza para estabilidade numérica

In [ ]:
# 5. Função personalizada para decision_function
def gb_decision_function(model, instance, initial_prediction):
    learning_rate = model.learning_rate
    estimators = model.estimators_

    estimator_results = []

    # Soma as previsões das árvores
    for estimator in estimators:
        tree_results = [tree.predict([instance])[0] for tree in estimator]
        estimator_results.append(tree_results)

    # Soma dos resultados das árvores e aplicação do learning rate
    estimator_sum = np.sum(estimator_results, axis=0) * learning_rate

    # Adiciona a previsão inicial ao somatório
    total_prediction = initial_prediction + estimator_sum

    return total_prediction

In [ ]:
# 7. Contabilizando acertos e divergências
def gb_test_decision_function(model, X, initial_prediction):
    equal = 0
    not_equal = 0

    for x in X:
        custom_decision = gb_decision_function(model, x, initial_prediction)
        sklearn_decision = model.decision_function([x])[0]

        if np.allclose(custom_decision, sklearn_decision):
            equal += 1
        else:
            not_equal += 1

    print(f"Iguais: {equal}")
    print(f"Diferentes: {not_equal}")

In [ ]:
# cancer = load_breast_cancer()
# X, y = cancer.data, cancer.target

In [ ]:
# wine = load_wine()
# X, y = wine.data, wine.target

# iris = load_iris()
# X, y = iris.data, iris.target

cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

In [ ]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [ ]:
gb_classifier.init_

DummyClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

# 3. Treinamento do modelo Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(random_state=42)
gb_classifier.fit(X_train, y_train)

initial_prediction = multiclass_initial_prediction(y_train, n_classes=len(gb_classifier.classes_))
print(initial_prediction)

# # 6. Testando a função personalizada
# for i, x in enumerate(X_test[:5]):  # Exibe os primeiros 5 exemplos
#     custom_decision = gb_decision_function(gb_classifier, x, initial_prediction)
#     sklearn_decision = gb_classifier.decision_function([x])[0]

#     print(f"Exemplo {i + 1}:")
#     print("Custom Decision Function:", custom_decision)
#     print("Sklearn Decision Function:", sklearn_decision)
#     print("Iguais?", np.allclose(custom_decision, sklearn_decision), "\n")



# 8. Avaliando a consistência
gb_test_decision_function(gb_classifier, X_test, initial_prediction)


[0.37362637 0.62637363]
[-0.98449898 -0.46780824]
final: [-0.25834537  0.25834537]
[-0.25834537  0.25834537]
Iguais: 0
Diferentes: 114


In [ ]:
gb_classifier.init_.class_prior_

array([0.37362637, 0.62637363])

In [ ]:
gb_classifier.decision_function(X_test)

array([ 7.23574555,  6.81026686,  7.76778977, -6.20070292,  6.79021177,
        7.46425776,  7.23399016, -7.67179804,  7.76778977,  7.23399016,
       -6.80620878,  6.09080399,  5.62086439,  7.6385289 , -7.39307454,
        5.95664537,  7.51772852,  7.39612985, -7.67179804,  2.30637853,
        7.05074303,  6.59538731,  7.20999748,  7.35479053, -7.67179804,
        2.89623175, -6.70596158,  7.91338112, -7.67179804, -4.87079675,
       -0.12074803, -6.50550904,  6.14179431,  0.09725034,  7.91338112,
        7.91338112, -4.71511567, -3.62761932, -7.47204364, -7.67179804,
       -7.55483146,  7.42175706,  7.35767578,  4.73088883,  7.54059135,
        7.3428526 , -7.47204364,  4.39153399, -3.73031431,  7.66252158,
       -7.12799615,  4.91656194,  6.01898181, -7.47204364,  7.51693023,
        7.33079786, -7.67179804, -7.67179804,  7.25507693,  7.51659867,
       -6.94122057, -0.59582244,  7.02009605,  7.76778977, -7.47204364,
        5.81266419,  6.20828672, -5.95835845, -5.73316836,  7.58

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [ ]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
# gb_classifier = GradientBoostingClassifier(n_estimators=2, learning_rate=0.01, max_depth=3, random_state=101, init="zero")
gb_classifier = GradientBoostingClassifier(n_estimators=2, learning_rate=0.01, max_depth=3, random_state=101)
gb_classifier.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.01, n_estimators=2, random_state=101)

In [ ]:
def multiclass_initial_prediction(y, n_classes):
    """Calcula a previsão inicial para cada classe no caso multiclasse."""
    class_counts = np.bincount(y, minlength=n_classes)
    class_probs = class_counts / len(y)  # Frequências das classes

    # Calcula a log odds para cada classe
    log_odds = np.log(class_probs + 1e-15)  # Evita log(0) com pequeno valor
    return log_odds - log_odds.mean()  # Centraliza para estabilidade numérica

def binary_initial_prediction(y):
    """Calcula a previsão inicial para classificação binária."""
    # Proporção de amostras positivas (classe 1)
    pi = np.mean(y)

    # Calcula a log odds (estimador inicial)
    initial_log_odds = np.log(pi / (1 - pi))

    return initial_log_odds

In [ ]:
if len(np.unique(y)) == 2:  # Binário
    initial_prediction = binary_initial_prediction(y)
else:  # Multiclasse
    initial_prediction = multiclass_initial_prediction(y, n_classes=len(gb_classifier.classes_))
initial_prediction

array([0., 0., 0.])

# Gradient Boost Predict

In [ ]:
def gb_decision_function(model, instance, init):
  learning_rate = gb_classifier.learning_rate

  estimators = model.estimators_
  count = 0
  estimator_results = []
  tree_results_return = []

  for estimator in estimators:
    count += 1
    class_number = 0
    tree_results = []

    for tree in estimator:
      tree_results.append((tree.predict([instance])[0]))
      class_number += 1

    estimator_results.append(tree_results)
  estimator_results.append(init)

  estimator_sum = np.sum(estimator_results, axis=0) * learning_rate
  return estimator_sum

def gb_test_decision_function(model, X, init):
  equal = 0
  not_equal = 0

  for x in X:
    estimator_sum = gb_decision_function(model, x, init)
    decision = model.decision_function([x])[0]

    if np.allclose(estimator_sum, decision):
      equal +=1
    else:
      not_equal +=1

  print("equal", equal)
  print("not equal", not_equal)


In [ ]:
estimator_sum = gb_decision_function(gb_classifier, X_test[0], initial_prediction)

In [ ]:
print(estimator_sum)
print(gb_classifier.decision_function([X_test[0]]))

[ 0.0396057  -0.01946811 -0.0204563 ]
[[ 0.04044007 -0.06992703  0.02916824]]


[ 0.03960594 -0.01995    -0.01995   ]

[[ 0.03960594 -0.01995    -0.01995   ]]

In [ ]:
print(estimator_sum)
print(gb_classifier.decision_function([X_test[0]]))

[ 0.0396057  -0.01946811 -0.0204563 ]
[[ 0.04044007 -0.06992703  0.02916824]]


In [ ]:
gb_test_decision_function(gb_classifier, X_test, initial_prediction)

equal 0
not equal 30


# grad boost

In [ ]:
gb_classifier = GradientBoostingClassifier(n_estimators=2, learning_rate=0.01, max_depth=3, random_state=101, init="zero")

In [ ]:
gb_classifier.fit(X_train, y_train)

GradientBoostingClassifier(init='zero', learning_rate=0.01, n_estimators=2,
                           random_state=101)

In [ ]:
y_pred = gb_classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667


In [ ]:
new_instance = X_test[0]  # Get the first instance from the test set
prediction = gb_classifier.predict([new_instance])
print("Real value:", y_test[0])
print("Prediction for the new instance:", prediction)

Real value: 0
Prediction for the new instance: [0]


In [ ]:
estimators = gb_classifier.estimators_
print(estimators)

[[DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                        random_state=RandomState(MT19937) at 0x793B22080A40)
  DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                        random_state=RandomState(MT19937) at 0x793B22080A40)
  DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                        random_state=RandomState(MT19937) at 0x793B22080A40)]
 [DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                        random_state=RandomState(MT19937) at 0x793B22080A40)
  DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                        random_state=RandomState(MT19937) at 0x793B22080A40)
  DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                        random_state=RandomState(MT19937) at 0x793B22080A40)]]


In [ ]:
estimators[0, 0].tree_.feature[0]

3

In [ ]:
# plt.figure(figsize=(8, 4))
# for estimator in estimators:
#   for tree in estimator:
#     plot_tree(tree, filled=True, feature_names=iris.feature_names, class_names=iris.target_names)
#     plt.show()

In [ ]:
for estimator in estimators:
  print()
  print("estimator")
  count = 0
  for tree in estimator:
    print("probability of class", count)
    count += 1
    print(tree.predict([new_instance]))


estimator
probability of class 0
[2.]
probability of class 1
[-1.]
probability of class 2
[-1.]

estimator
probability of class 0
[1.96059404]
probability of class 1
[-0.99500037]
probability of class 2
[-0.99500037]


In [ ]:
gb_classifier.init_.class_prior_

AttributeError: 'str' object has no attribute 'class_prior_'

In [ ]:
# prompt: use the decision function
decision_function_values = gb_classifier.decision_function(X_test)
print("Decision function values:", decision_function_values)

Decision function values: [[ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [-1.99509565e-02  9.94849538e-03 -1.99750005e-02]
 [-1.99509565e-02  3.79633068e-02 -1.99513923e-02]
 [-1.99509565e-02 -1.99500037e-02  3.96699134e-02]
 [-1.99428587e-02  3.79633068e-02 -1.99513923e-02]
 [-1.99509565e-02  3.79633068e-02 -1.99513923e-02]
 [-1.99509565e-02 -1.99500037e-02  3.96699134e-02]
 [ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [-1.99509565e-02 -1.99500037e-02  3.96059404e-02]
 [ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [-1.99509565e-02 -1.99500037e-02  3.96059404e-02]
 [-1.99509565e-02 -1.99500037e-02 -2.76422367e-05]
 [-1.99509565e-02  3.79633068e-02 -1.99513923e-02]
 [-1.99509565e-02  3.79633068e-02 -1.99513923e-02]
 [-1.99509565e-02  3.79633068e-02 -1.99513923e-02]
 [ 3.96059404e-02 -1.99500037e-02 -1.99500037e-02]
 [-1.

In [ ]:
# prompt: make  a function that receibes the fited model and makes a prediction without .predict

def custom_predict(model, X):
  """
  Makes a prediction using a fitted GradientBoostingClassifier model without using .predict.

  Args:
    model: The fitted GradientBoostingClassifier model.
    X: The input data for prediction.

  Returns:
    A list of predicted classes.
  """
  predictions = []
  for x in X:
    # Get the decision function values for each class
    decision_function_values = model.decision_function([x])[0]

    # Find the class with the highest decision function value
    predicted_class = decision_function_values.argmax()
    predictions.append(predicted_class)

  return predictions

# Example usage:
new_instance = X_test[0]
custom_prediction = custom_predict(gb_classifier, [new_instance])
print("Real value:", y_test[0])
print("Custom prediction for the new instance:", custom_prediction)


Real value: 0
Custom prediction for the new instance: [0]


In [ ]:
set(iris.target)

{0, 1, 2}

In [ ]:
def my_predict(model, X):
  estimator = model.estimators_
  count = 0
  estimator_results = []
  tree_results_return = []

  for estimator in estimators:
    print("\nestimator", count)
    count += 1
    class_number = 0
    tree_results = []

    for tree in estimator:
      print("proba of class", class_number)
      print(tree.predict([X]))
      tree_results.append((class_number, tree.predict([X])[0]))
      tree_results_return.append(tree.predict([X])[0])
      class_number += 1

    max_probability = max([x[1] for x in tree_results])
    for class_number, probability in tree_results:
      if probability == max_probability:
        print("Class with highest probability:", class_number)
        estimator_results.append((class_number, probability))
        break

  class_numbers = [x[0] for x in estimator_results]
  most_common_class = Counter(class_numbers).most_common(1)[0][0]
  print("\nPrediction =", most_common_class)

  result = most_common_class
  return result, estimator_results, tree_results_return

In [ ]:
instance = X_test[0]
instance

array([5.5, 4.2, 1.4, 0.2])

In [ ]:
result, estimator_results, tree_result = my_predict(gb_classifier, instance)


estimator 0
proba of class 0
[2.]
proba of class 1
[-1.]
proba of class 2
[-1.]
Class with highest probability: 0

estimator 1
proba of class 0
[1.96059404]
proba of class 1
[-0.99500037]
proba of class 2
[-0.99500037]
Class with highest probability: 0

Prediction = 0


In [ ]:
print("Decision function values:", decision_function_values[0])

Decision function values: [ 0.03960594 -0.01995    -0.01995   ]


In [ ]:
result

0

In [ ]:
estimator_results

[(0, 2.0), (0, 1.9605940447313444)]

In [ ]:
tree_result

[2.0,
 -0.9999999999999999,
 -0.9999999999999997,
 1.9605940447313444,
 -0.9950003749662529,
 -0.9950003749662529]

In [ ]:
tree_result = np.array(tree_result, dtype=np.float32)
tree_result = tree_result * 0.01
tree_result

array([ 0.02      , -0.01      , -0.01      ,  0.01960594, -0.00995   ,
       -0.00995   ], dtype=float32)

In [ ]:
tree_matrix = np.array(tree_result).reshape(2, 3)
tree_matrix

array([[ 0.02      , -0.01      , -0.01      ],
       [ 0.01960594, -0.00995   , -0.00995   ]], dtype=float32)

In [ ]:
column_sums = np.sum(tree_matrix, axis=0)
result = column_sums
result

array([ 0.03960594, -0.01995   , -0.01995   ], dtype=float32)

In [ ]:
decision_function_instance = gb_classifier.decision_function([instance])
decision_function_instance

array([[ 0.03960594, -0.01995   , -0.01995   ]])

In [ ]:
gb_classifier.predict_proba([instance])

array([[0.34669668, 0.32665166, 0.32665166]])

In [ ]:
importances = gb_classifier.feature_importances_
importances

array([3.25600024e-08, 9.97226297e-03, 1.15952239e-01, 8.74075465e-01])

In [ ]:
gb_classifier.init_.class_prior_

AttributeError: 'str' object has no attribute 'class_prior_'

In [ ]:
score = np.zeros((X_test.shape[0], 3)) + gb_classifier.init_.class_prior_

In [ ]:
for stage in gb_classifier.estimators_:
        # Cada 'stage' é uma lista de árvores (uma por classe)
        for k, tree in enumerate(stage):
            # Soma as previsões da árvore ponderadas pela learning rate
            score[:, k] += 0.01 * tree.predict(X_test)

In [ ]:
score

# test

In [ ]:
ok_count = 0
for x in X_test:
  gbpred = gb_classifier.predict([x])[0]
  mypred = my_predict(gb_classifier, x)[0]
  if gbpred != mypred:
    print("\nerror")
    print("gb predict =", gbpred)
    print("my predict =", mypred)
  else:
    ok_count += 1

print(ok_count)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
import numpy as np

# 3. Função para calcular o valor inicial (bias)
def get_initial_score(y, n_classes):
    if n_classes == 2:
        p = np.clip(np.mean(y), 1e-15, 1 - 1e-15)  # Evita problemas numéricos
        return np.log(p / (1 - p))  # Log-odds para binário
    else:
        class_counts = np.bincount(y, minlength=n_classes) / len(y)
        print(class_counts)
        print(np.log(class_counts))
        return np.log(class_counts)  # Log das probabilidades para multiclasse

# 4. Função para calcular manualmente a decision_function
def manual_decision_function(X, gb_classifier):
    n_classes = gb_classifier.n_classes_
    learning_rate = gb_classifier.learning_rate
    init_score = get_initial_score(y, n_classes)

    if X.ndim == 1:
        X = X.reshape(1, -1)


    # Inicializa as pontuações com o valor inicial (bias)
    if n_classes == 2:
        score = np.full(X.shape[0], init_score)  # Binário: array 1D
    else:
        score = np.tile(init_score * learning_rate, (X.shape[0], 1))  # Multiclasse: array 2D
    # Itera sobre os estágios (listas de árvores por classe)
    for i, stage in enumerate(gb_classifier.estimators_):
        for k, tree in enumerate(stage):
            if n_classes == 2:
                # Para binário, apenas uma árvore por estágio (para a classe positiva)
                score += learning_rate * tree.predict(X).ravel()
            else:
                # Para multiclasse, cada árvore prediz uma classe específica
                score[:, k] += learning_rate * tree.predict(X).ravel()

    return score

# 5. Cálculo manual da decision_function
manual_scores = manual_decision_function(X_test[0], gb_classifier)

# 6. Comparação com a decisão real do gb_classifiero
original_scores = gb_classifier.decision_function([X_test[0]])

print("Manual Decision Function:\n", manual_scores)
print("\nOriginal Decision Function:\n", original_scores)

# Verificando se os resultados são próximos
print("\nDiferença média:", np.mean(np.abs(manual_scores - original_scores)))


In [ ]:
init = gb_classifier.init_
raw_init_score = init.predict(X_test)

In [ ]:
init.predict_proba(X_test)

In [ ]:
raw_init_score
raw_init_score.ravel()